In [1]:
!pip install findspark
!pip install pyspark
!pip install spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=4ebc868436af1c472696fb013f5605c6a52d3ba3547d26d4f8dddda0572f69cb
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58748 sha256=5358fd4e7e798719d61c3e07777cca63daa1859a0ce5b3e118acb5d596726c59
  Stored in directory: /root/.cache/pip/wheels/63/88/77/b4131110ea4094540f7b47c6d62a649807d7e94800da5eab0b
Successfully built spark


In [2]:
import pyspark
import findspark
import spark

findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

In [15]:
df = spark.read.parquet('dataframe.parquet')

# Particiones y almacenado en archivo

In [18]:
df1 = df.repartition(2)

In [20]:
df1.write.format('csv').option('sep', '|').save('./output/csv')

In [21]:
df1.coalesce(1).write.format('csv').option('sep', '|').save('./output/csv1')

In [22]:
df.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- trending_date: string (nullable = true)
 |-- title: string (nullable = true)
 |-- channel_title: string (nullable = true)
 |-- category_id: string (nullable = true)
 |-- publish_time: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- views: string (nullable = true)
 |-- likes: string (nullable = true)
 |-- dislikes: string (nullable = true)
 |-- comment_count: string (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: string (nullable = true)
 |-- ratings_disabled: string (nullable = true)
 |-- video_error_or_removed: string (nullable = true)
 |-- description: string (nullable = true)



In [24]:
df.select('comments_disabled').distinct().show()

+-----------------+
|comments_disabled|
+-----------------+
|            False|
|             null|
| sports and more.|
|          Wiz Kid|
|             True|
|         farfalle|
+-----------------+



In [27]:
df_limpio = df.filter(col('comments_disabled').isin('False', 'True'))

In [28]:
df_limpio.write.partitionBy('comments_disabled').parquet('./output/parquet')

# Persistir DF

In [29]:
df = spark.createDataFrame([(1,'a'), (2,'b'), (3,'c'), (4,'d')], ['id', 'valor'])

In [30]:
df.show()

+---+-----+
| id|valor|
+---+-----+
|  1|    a|
|  2|    b|
|  3|    c|
|  4|    d|
+---+-----+



In [31]:
df.persist()

DataFrame[id: bigint, valor: string]

In [32]:
df.unpersist()

DataFrame[id: bigint, valor: string]

In [33]:
df.cache()

DataFrame[id: bigint, valor: string]

In [36]:
from pyspark.storagelevel import StorageLevel

In [37]:
df.persist(StorageLevel.DISK_ONLY)

DataFrame[id: bigint, valor: string]

# EJERCICIO 5

In [5]:
cases1 = spark.read.option('header','true').option('inferSchema', 'true').csv('Case.csv')

In [39]:
cases1.printSchema()

root
 |--  case_id: integer (nullable = true)
 |-- province: string (nullable = true)
 |-- city: string (nullable = true)
 |-- group: boolean (nullable = true)
 |-- infection_case: string (nullable = true)
 |-- confirmed: integer (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)



In [7]:
cases.orderBy(col('confirmed').cast('int').desc()).select('province', 'city', 'confirmed').show(3)

+--------+------+---------+
|province|  city|confirmed|
+--------+------+---------+
|   Daegu|Nam-gu|     4511|
|   Daegu|     -|      917|
|   Daegu|     -|      747|
+--------+------+---------+
only showing top 3 rows



In [7]:
patients = spark.read.option('header','true').option('inferSchema','true').csv('/content/PatientInfo.csv')

In [9]:
'''df_patients = patients.select(
    col('patient_id').cast('int'),
    'sex',
    'age',
    'country',
    'province',
    'city',
    'infection_case',
    col('infected_by').cast('int'),
    col('contact_number').cast('int'),
    col('symptom_onset_date').cast('date'),
    col('confirmed_date').cast('date'),
    col('released_date').cast('date'),
    col('deceased_date').cast('date'),
    'state'
)'''

patients.count()

5165

In [14]:
patients_nd = patients.dropDuplicates(['patient_id'])
patients_nd.count()

5164

In [18]:
df_infectedBy = patients_nd.filter(col('infected_by').isNotNull())

In [32]:
infected_female = df_infectedBy.filter(col('sex')  == 'female').drop('released_date','deceased_date')

In [33]:
infected_female.show()

+----------+------+---+-------+-----------------+-------------+--------------------+-----------+--------------+------------------+--------------+--------+
|patient_id|   sex|age|country|         province|         city|      infection_case|infected_by|contact_number|symptom_onset_date|confirmed_date|   state|
+----------+------+---+-------+-----------------+-------------+--------------------+-----------+--------------+------------------+--------------+--------+
|1000000046|female|20s|  Korea|            Seoul|          etc|contact with patient| 1000000041|          null|              null|    2020-02-26|released|
|1500000059|female|50s|  Korea|          Daejeon|       Sejong|contact with patient| 1500000049|             6|              null|    2020-06-17|isolated|
|2000000108|female|10s|  Korea|      Gyeonggi-do|  Hwaseong-si|contact with patient| 2000000104|          null|              null|    2020-03-04|isolated|
|2000000390|female|40s|  Korea|      Gyeonggi-do|Pyeongtaek-si|contact

In [36]:
infected_female.write.mode('overwrite').partitionBy('province').parquet('output/infected_female_province')

# AGREGACIONES

In [7]:
df.printSchema()

root
 |-- nombre: string (nullable = true)
 |-- color: string (nullable = true)
 |-- cantidad: long (nullable = true)



In [8]:
df.show()

+------+-----+--------+
|nombre|color|cantidad|
+------+-----+--------+
|  Jose| azul|    1900|
|  null| null|    1700|
|  null| rojo|    1300|
|  Juan| rojo|    1500|
+------+-----+--------+



In [9]:
# Count

df.count()

4

In [11]:
# count no cuenta nulos

df.select(
    count('nombre').alias('conteo_nombre'),
    count('color').alias('conteo_colores'),
    count('*').alias('conteo_general')
).show()

+-------------+--------------+--------------+
|conteo_nombre|conteo_colores|conteo_general|
+-------------+--------------+--------------+
|            2|             3|             4|
+-------------+--------------+--------------+



In [16]:
# countDistinct devuelve los valores distintos sin tomar en cuenta nulos

df.select(
    countDistinct('color').alias('colores_dif')
).show()

+-----------+
|colores_dif|
+-----------+
|          2|
+-----------+



In [20]:
# approx_count_distinct

#dataframe = spark.read.parquet('/content/vuelos.parquet')
df_vuelos = spark.read.option('header','true').parquet('vuelos.parquet')

In [22]:
df_vuelos.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

In [23]:
df_vuelos.select(
    countDistinct('AIRLINE'),
    approx_count_distinct('AIRLINE')
).show()

+-----------------------+------------------------------+
|count(DISTINCT AIRLINE)|approx_count_distinct(AIRLINE)|
+-----------------------+------------------------------+
|                     14|                            13|
+-----------------------+------------------------------+



In [24]:
df_vuelos.select(
    max('AIR_TIME'),
    min('AIR_TIME')
).show()

+-------------+-------------+
|max(AIR_TIME)|min(AIR_TIME)|
+-------------+-------------+
|          690|            7|
+-------------+-------------+



In [27]:
df_vuelos.select(
    min('AIRLINE_DELAY'),
    max('AIRLINE_DELAY')
).show()

+------------------+------------------+
|min(AIRLINE_DELAY)|max(AIRLINE_DELAY)|
+------------------+------------------+
|                 0|              1971|
+------------------+------------------+



In [29]:
df_vuelos.select(
    sum('DISTANCE')
).show()

+-------------+
|sum(DISTANCE)|
+-------------+
|   4785357409|
+-------------+



In [32]:
df_vuelos.select(
    sum_distinct('DISTANCE')
).show()

+----------------------+
|sum(DISTINCT DISTANCE)|
+----------------------+
|               1442300|
+----------------------+



In [35]:
df_vuelos.select(
    avg('AIR_TIME'),
    (sum('AIR_TIME') / count('AIR_TIME')).alias('promedio_manual')
).show()

+------------------+------------------+
|     avg(AIR_TIME)|   promedio_manual|
+------------------+------------------+
|113.51162809012519|113.51162809012519|
+------------------+------------------+



# Agrupaciones

In [55]:
df_vuelos.groupBy('ORIGIN_AIRPORT', 'DESTINATION_AIRPORT')\
      .agg(
          count(col('*')).alias('conteo_aeropuertos')
          )\
           .orderBy(desc('conteo_aeropuertos')).show()

+--------------+-------------------+------------------+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|conteo_aeropuertos|
+--------------+-------------------+------------------+
|           SFO|                LAX|             13744|
|           LAX|                SFO|             13457|
|           JFK|                LAX|             12016|
|           LAX|                JFK|             12015|
|           LAS|                LAX|              9715|
|           LGA|                ORD|              9639|
|           LAX|                LAS|              9594|
|           ORD|                LGA|              9575|
|           SFO|                JFK|              8440|
|           JFK|                SFO|              8437|
|           OGG|                HNL|              8313|
|           HNL|                OGG|              8282|
|           LAX|                ORD|              8256|
|           ATL|                LGA|              8234|
|           LGA|                ATL|            

In [56]:
df_vuelos.groupBy('ORIGIN_AIRPORT')\
      .agg(
          count('AIR_TIME').alias('tiempo_aire'),
          min('AIR_TIME').alias('minimo_timepo'),
          max('AIR_TIME').alias('max_tiempo')
          )\
           .orderBy(desc('tiempo_aire')).show()

+--------------+-----------+-------------+----------+
|ORIGIN_AIRPORT|tiempo_aire|minimo_timepo|max_tiempo|
+--------------+-----------+-------------+----------+
|           ATL|     343506|           15|       614|
|           ORD|     276554|           13|       571|
|           DFW|     232647|           11|       534|
|           DEN|     193402|           12|       493|
|           LAX|     192003|           14|       409|
|           PHX|     145552|           19|       444|
|           SFO|     145491|            8|       389|
|           IAH|     144019|           15|       524|
|           LAS|     131937|           25|       429|
|           MSP|     111055|           14|       537|
|           SEA|     110178|           17|       412|
|           MCO|     109532|           25|       395|
|           DTW|     106992|           15|       341|
|           BOS|     104804|           16|       432|
|           CLT|      99052|           17|       379|
|           EWR|      98341|

In [57]:
df_vuelos.groupBy('MONTH').\
agg(
    count('ARRIVAL_DELAY').alias('retraso'),
    avg('DISTANCE').alias('avg_distancia')
).orderBy(desc('retraso')).show()

+-----+-------+-----------------+
|MONTH|retraso|    avg_distancia|
+-----+-------+-----------------+
|    7| 514384|841.4772794487611|
|    8| 503956|834.8244276603413|
|    6| 492847|835.6302716626612|
|    3| 492138|816.0553268611494|
|    5| 489641|823.3230588760807|
|   10| 482878|816.4436127652134|
|    4| 479251|817.0060476016745|
|   12| 469717|837.8018926194103|
|   11| 462367|820.2482434846529|
|    9| 462153|815.8487523282274|
|    1| 457013|803.2612794913696|
|    2| 407663| 800.785449834689|
+-----+-------+-----------------+



# Pivot

In [59]:
estudiantes = spark.read.parquet('estudiantes.parquet')

In [61]:
estudiantes.show()

+------+----+----+----------+
|nombre|sexo|peso|graduacion|
+------+----+----+----------+
|  Jose|   M|  80|      2000|
| Hilda|   F|  50|      2000|
|  Juan|   M|  75|      2000|
| Pedro|   M|  76|      2001|
|Katia+|   F|  65|      2001|
+------+----+----+----------+



In [63]:
# promedio de peso por año de graduacion y sexo

estudiantes.groupBy('graduacion').pivot('sexo').avg('peso').show()

+----------+----+----+
|graduacion|   F|   M|
+----------+----+----+
|      2001|65.0|76.0|
|      2000|50.0|77.5|
+----------+----+----+



In [67]:
estudiantes.groupBy('graduacion').pivot('sexo').agg(avg('peso').alias('promedio'), min('peso').alias('min_peso'), max('peso').alias('max_peso')).show()

+----------+----------+----------+----------+----------+----------+----------+
|graduacion|F_promedio|F_min_peso|F_max_peso|M_promedio|M_min_peso|M_max_peso|
+----------+----------+----------+----------+----------+----------+----------+
|      2001|      65.0|        65|        65|      76.0|        76|        76|
|      2000|      50.0|        50|        50|      77.5|        75|        80|
+----------+----------+----------+----------+----------+----------+----------+



In [76]:
estudiantes.groupBy('graduacion')\
.pivot('sexo', ['M'])\
.agg(
    avg('peso').alias('promedio'),
    min('peso').alias('min_peso'),
    max('peso').alias('max_peso')
    )\
.show()

+----------+----------+----------+----------+
|graduacion|M_promedio|M_min_peso|M_max_peso|
+----------+----------+----------+----------+
|      2001|      76.0|        76|        76|
|      2000|      77.5|        75|        80|
+----------+----------+----------+----------+

